### **Classe Rede Neural**

### **Imports**

In [ ]:
import numpy as np # biblioteca de manipulção matemática

In [ ]:
from sklearn.datasets import fetch_openml # para baixar dataset
from sklearn.model_selection import train_test_split # para dividir dataset em treino e validação
from keras.utils import to_categorical

### **Rede Neural**

In [ ]:
import numpy as np

class RedeNeuralProfunda():
  """
  Classe para a rede neural profunda.
  """
  def __init__(self, arquitetura, caminho_modelo=None, epocas=10, taxa_aprendizado=0.05):
    """
    Inicializa a rede neural profunda.

    Parâmetros:
    - arquitetura: lista com a arquitetura da rede
    - caminho_modelo: caminho para o arquivo de modelo (matrizes numpy .npz)
    - epocas: número de épocas de treinamento
    - taxa_aprendizado: taxa de aprendizado
    """
    self.arquitetura = arquitetura
    self.num_camadas = len(self.arquitetura)
    self.epocas = epocas
    self.taxa_aprendizado = taxa_aprendizado
    self.parametros = {}

    # Carregar pesos:
    if caminho_modelo:
      pesos = np.load(caminho_modelo, allow_pickle=True) # pesos em arquivo .npz
      for chave, valor in pesos.items():
        self.parametros[chave] = valor
    else:
      self.inicializarPesos()

    # Inicializar matrizes vazias:
    for i in range(self.num_camadas):
      if not i == 0:
        self.parametros[f"Z{i}"] = None # armazenar somas ponderadas
      self.parametros[f"N{i}"] = None # armazenar valores das ativações

  def inicializarPesos(self, semente=42):
    """
    Inicializa os pesos da rede neural.
    """
    np.random.seed(semente)

    for i in range(1, self.num_camadas):
      self.parametros[f"P{i}"] = np.random.randn(self.arquitetura[i], self.arquitetura[i-1]) # inicializa pesos para cada camada

  def salvarPesos(self, caminho="pesos.npz"):
    """
    Salva os pesos da rede neural em um arquivo .npz.

    Parâmetros:
    - caminho: caminho para o arquivo de saída
    """
    np.savez(caminho, **self.parametros)

  def sigmoid(self, x, derivada=False):
    """
    Função de ativação sigmoid.

    Parâmetros:
    - x: entrada
    - derivada: se True, retorna a derivada da função sigmoid
    """
    if derivada:
      return (np.exp(-x))/((np.exp(-x)+1)**2)
    return 1/(1 + np.exp(-x))

  def softmax(self, x, derivada=False):
    """
    Função de ativação softmax.

    Parâmetros:
    - x: entrada
    - derivada: se True, retorna a derivada da função softmax
    """
    exps = np.exp(x - x.max())
    if derivada:
      return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))

  def propogar(self, entrada):
    """
    Propaga a entrada para a rede neural.

    Parâmetros:
    - entrada: entrada da rede
    """
    parametros = self.parametros

    parametros["N0"] = entrada

    for cont in range(1, self.num_camadas):
      parametros[f"Z{cont}"] = np.dot(parametros[f"P{cont}"], parametros[f"N{cont-1}"])
      parametros[f"N{cont}"] = self.sigmoid(parametros[f"Z{cont}"])

    return parametros[f"N{self.num_camadas - 1}"]

  def retroPropagar(self, saida, target):
    """
    Retro-propaga o erro para a rede neural.

    Parâmetros:
    - saida: saída da rede
    - target: target da rede
    """
    parametros = self.parametros
    ajustar_pesos = {}

    erro = 2 * (target - saida) / target.shape[0] * self.softmax(self.parametros[f"Z{self.num_camadas - 1}"], derivada=True)
    ajustar_pesos[f"P{self.num_camadas - 1}"] = np.outer(erro, parametros[f"N{self.num_camadas - 2}"])

    for cont in range(self.num_camadas - 1, 2, -1):
      erro = np.dot(parametros[f"P{cont}"].T, erro) * self.sigmoid(parametros[f"Z{cont - 1}"], derivada=True)
      ajustar_pesos[f"P{cont - 1}"] = np.outer(erro, parametros[f"N{cont - 2}"])

    return ajustar_pesos

  def atualizarParametros(self, ajustar_para_pesos):
    """
    Atualiza os pesos da rede neural.

    Parâmetros:
    - ajustar_para_pesos: dicionário com os pesos a serem atualizados
    """
    for chave, valor in ajustar_para_pesos.items():
      self.parametros[chave] -= self.taxa_aprendizado * valor

  def calcularAcuracia(self, x_entrada, y_saida):
    """
    Calcula a acurácia da rede neural.

    Parâmetros:
    - x_entrada: entradas da rede
    - y_saida: saidas
    """
    predicao = []

    for x, y in zip(x_entrada, y_saida):
      saida = self.propogar(x)
      pred = np.argmax(saida)
      predicao.append(pred == np.argmax(y))

    return np.mean(predicao)

  def treinamento(self, x_treino, y_treino, x_teste, y_teste):
    """
    Treina a rede neural.

    Parâmetros:
    - x_treino: entradas de treinamento
    - y_treino: saidas de treinamento
    - x_teste: entradas de teste
    - y_teste: saidas de teste
    """
    for interacao in range(self.epocas):
      for x, y in zip(x_treino, y_treino):
        saida = self.propogar(x)
        ajustar_para_pesos = self.retroPropagar(y, saida)
        self.atualizarParametros(ajustar_para_pesos)

      acuracia = self.calcularAcuracia(x_teste, y_teste)
      print(f"Época: {interacao+1} | Acurácia: {acuracia * 100}")


### **Instanciando modelo**

In [ ]:
rede = RedeNeuralProfunda([100, 500, 400, 500, 500, 100, 800, 5], epocas=10)

## **Treinando com MNIST🏃**

### **Instaciando o Modelo**

In [ ]:
28*28

784

In [ ]:
rede = RedeNeuralProfunda([784, 100, 50, 10], epocas=5)

### **Obtendo os Dados**

In [ ]:
x, y = fetch_openml('mnist_784', version=1, return_X_y=True, parser='auto')

In [ ]:
x.shape, y.shape

((70000, 784), (70000,))

### **Preparando os Dados**

In [ ]:
x = (x/255).astype('float32') # normalização

In [ ]:
y = to_categorical(y) # one-hot encoding

In [ ]:
# 0 → [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# 1 → [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
# 2 → [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# 3 → [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
# 4 → [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
# 5 → [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
# 6 → [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
# 7 → [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
# 8 → [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
# 9 → [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=22) # dividir em treino e teste

In [ ]:
# Convertendo para numpy array:
x_train = x_train.to_numpy()
x_val = x_val.to_numpy()

### **Treinando o Modelo**

In [ ]:
rede.treinamento(x_train, y_train, x_val, y_val)

Época: 1 | Acurácia: 52.97857142857143
Época: 2 | Acurácia: 63.464285714285715
Época: 3 | Acurácia: 68.8
Época: 4 | Acurácia: 72.94285714285714
Época: 5 | Acurácia: 75.5142857142857


### **Salvando Modelo**

In [ ]:
rede.salvarPesos()

### **E se quisermos acelerar?**

In [ ]:
import cupy as np
import numpy as np
import time

N = 10000 # tamanho da matriz

# criando matrizes aleatórias:
matrix_a_cpu = np.random.rand(N, N).astype(np.float32)
matrix_b_cpu = np.random.rand(N, N).astype(np.float32)

# tempo de multiplicação na CPU:
start_time_cpu = time.time()
result_cpu = np.dot(matrix_a_cpu, matrix_b_cpu)
end_time_cpu = time.time()
cpu_time = end_time_cpu - start_time_cpu

# transferindo matrizes para GPU:
matrix_a_gpu = cp.array(matrix_a_cpu)
matrix_b_gpu = cp.array(matrix_b_cpu)

# tempo de multiplicação na GPU:
start_time_gpu = time.time()
result_gpu = cp.dot(matrix_a_gpu, matrix_b_gpu)
end_time_gpu = time.time()
gpu_time = end_time_gpu - start_time_gpu

# convertendo o resultado da GPU para CPU (opcional):
result_gpu_cpu = cp.asnumpy(result_gpu)

print(f"Tempo de execução na CPU: {cpu_time:.6f} segundos")
print(f"Tempo de execução na GPU: {gpu_time:.6f} segundos")

Tempo de execução na CPU: 16.285794 segundos
Tempo de execução na GPU: 0.001233 segundos
